In [62]:
from siml.sk_utils import *
from siml.signal_analysis_utils import *
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pywt
import scipy.stats
import pandas as pd
from collections import defaultdict, Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

from sklearn.metrics import classification_report
from detecta import detect_peaks
from scipy.fftpack import fft
from scipy.signal import welch
from sklearn.model_selection import train_test_split  

## 加载数据文件

In [63]:
typeDescription = {
    0: 'normal',
    1: 'pothole',
    2: 'transverse',
}

def readFile(filename):
    return np.loadtxt(filename)

In [64]:
folderPath = '../../data/Final_Version/all/datasets/'
dataFile = ['dataX.txt', 'dataY.txt', 'dataZ.txt']
labelFile = ['dataLabel.txt']

signals = []
for file in dataFile:
    dataPath = folderPath+file
    signals.append(np.loadtxt(dataPath))
signals = np.transpose(np.array(signals), (1, 2, 0))
print('数据集：', signals.shape)

labelFilePath = folderPath+labelFile[0]
dataLabel = np.loadtxt(labelFilePath)
anomalyType = list(dataLabel[:,0])

dic = {}
temp = Counter(anomalyType)
for key in temp.keys():
    dic[typeDescription[key]] = temp[key]
print(dic)

数据集： (4088, 64, 3)
{'normal': 3601, 'pothole': 474, 'transverse': 13}


## 特征提取

### 频域特征

In [65]:
N = 64 #样本数
f_s = 50 #采样频率
denominator = 10

#时域
def get_values(y_values, N, f_s):
    f_values = np.linspace(0.0, N/f_s, N)
    return f_values, y_values

#FFT
def get_fft_values(y_values, N, f_s):
    f_values = np.linspace(0.0, f_s/2.0, N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values

#PSD
def get_psd_values(y_values, N, f_s):
    f_values, psd_values = welch(y_values, fs=f_s)
    return f_values, psd_values

#Autocorrelation
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]
 
def get_autocorr_values(y_values, N, f_s):
    autocorr_values = autocorr(y_values)
    x_values = np.array([ 1.0*jj/f_s for jj in range(0, N)])
    return x_values, autocorr_values


def get_first_n_peaks(x, y, no_peaks=5):
    x_, y_ = list(x), list(y)
    if len(x_) >= no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:#少于5个peaks，以0填充
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks


def get_features1(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(
        x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y


def extract_features1(dataset, N, f_s, denominator):
    percentile = 5
    list_of_features = []

    for signal_no in range(0, len(dataset)):
        features = [] #5*2*3*3
        
        for signal_comp in range(0, dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]

            signal_min = np.nanpercentile(signal, percentile)
            signal_max = np.nanpercentile(signal, 100-percentile)
            #ijk = (100 - 2*percentile)/10
            #set minimum peak height
            mph = signal_min + (signal_max - signal_min)/denominator

            features += get_features1(*get_psd_values(signal, N, f_s), mph)
            features += get_features1(*get_fft_values(signal, N, f_s), mph)
            features += get_features1(*get_autocorr_values(signal, N, f_s), mph)
        list_of_features.append(features)
    return np.array(list_of_features)

In [66]:
features1 = extract_features1(
    signals, N, f_s, denominator)

### 小波域特征

In [67]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    # print(counter_values)
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    # print(probabilities)
    entropy = scipy.stats.entropy(probabilities)
    return entropy


def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]


def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(
        np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]


def get_features2(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics


def extract_features2(dataset, waveletname):
    uci_har_features = []
    for signal_no in range(0, len(dataset)):
        features = []
        for signal_comp in range(0, dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            # 小波变换，返回长度为5，元素为单列array的list
            list_coeff = pywt.wavedec(signal, waveletname)
            #print(len(list_coeff))
            for coeff in list_coeff:
                # 对小波变换后的系数取其特征：entropy+statistics+crossings
                features += get_features2(coeff)
                #print(len(features))
        uci_har_features.append(features)
    X = np.array(uci_har_features)
    return X

In [68]:
waveletname = 'rbio3.1'
features2 = extract_features2(signals,waveletname)

In [69]:
print(features1.shape)
print(features2.shape)
features = np.hstack((features1,features2))
print(features.shape)
labels = np.array(anomalyType)

(4088, 90)
(4088, 180)
(4088, 270)


In [70]:
StandardScaler().fit_transform(features)
Normalizer().fit_transform(features)

array([[0.02863046, 0.04771743, 0.0763479 , ..., 0.01433826, 0.01682968,
        0.01083583],
       [0.03640027, 0.10010075, 0.1456011 , ..., 0.01505635, 0.01946188,
        0.01116774],
       [0.03780414, 0.06615724, 0.09451035, ..., 0.01879833, 0.02921117,
        0.0126046 ],
       ...,
       [0.02100236, 0.04200472, 0.06300707, ..., 0.01634429, 0.02981091,
        0.012177  ],
       [0.01916063, 0.08622285, 0.12454412, ..., 0.01677544, 0.0229487 ,
        0.01196653],
       [0.02907852, 0.08723557, 0.14539261, ..., 0.01156783, 0.01078556,
        0.00923979]])

## 信号分类

In [71]:
def randomize(features, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_features = features[permutation, :]
    shuffled_labels = labels[permutation]
    return shuffled_features, shuffled_labels

iteration = 100
warnings.filterwarnings('ignore')

### Logistic Regression

In [72]:
yTest, yPredict = [], []
trainingScore, testingScore = 0, 0
for i in range(iteration):
    X, Y = randomize(features,labels)
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
    clf = LogisticRegression(random_state=0)
    clf.fit(X_train, Y_train)
    Y_test_pred = clf.predict(X_test)
    yTest.extend(Y_test)
    yPredict.extend(Y_test_pred)
    trainingScore += clf.score(X_train, Y_train)
    testingScore += clf.score(X_test, Y_test)
print("Results of Logistic Regression")
print("Accuracy on training set is : {}".format(trainingScore/iteration))
print("Accuracy on test set is : {}".format(testingScore/iteration))
print(classification_report(yTest, yPredict,digits=3))

Results of Logistic Regression
Accuracy on training set is : 0.9668647326109747
Accuracy on test set is : 0.9471801140994297
              precision    recall  f1-score   support

         0.0      0.964     0.979     0.971    107974
         1.0      0.817     0.733     0.773     14330
         2.0      0.006     0.003     0.004       396

    accuracy                          0.947    122700
   macro avg      0.596     0.572     0.583    122700
weighted avg      0.944     0.947     0.945    122700



### Support Vector Machine

In [73]:
yTest, yPredict = [], []
trainingScore, testingScore = 0, 0
for i in range(iteration):
    X, Y = randomize(features,labels)
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
    clf = svm.SVC()
    clf.fit(X_train, Y_train)
    Y_test_pred = clf.predict(X_test)
    yTest.extend(Y_test)
    yPredict.extend(Y_test_pred)
    trainingScore += clf.score(X_train, Y_train)
    testingScore += clf.score(X_test, Y_test)
print("Results of Support Vector Machine")  
print("Accuracy on training set is : {}".format(trainingScore/iteration))
print("Accuracy on test set is : {}".format(testingScore/iteration))
print(classification_report(yTest, yPredict,digits=3))

Results of Support Vector Machine
Accuracy on training set is : 0.954267738552953
Accuracy on test set is : 0.9512469437652815
              precision    recall  f1-score   support

         0.0      0.955     0.993     0.973    108039
         1.0      0.914     0.663     0.769     14291
         2.0      0.000     0.000     0.000       370

    accuracy                          0.951    122700
   macro avg      0.623     0.552     0.581    122700
weighted avg      0.947     0.951     0.947    122700



### Random Forest

In [74]:
yTest, yPredict = [], []
trainingScore, testingScore = 0, 0
for i in range(iteration):
    X, Y = randomize(features,labels)
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, Y_train)
    Y_test_pred = clf.predict(X_test)
    yTest.extend(Y_test)
    yPredict.extend(Y_test_pred)
    trainingScore += clf.score(X_train, Y_train)
    testingScore += clf.score(X_test, Y_test)
print("Results of RandomForest")
print("Accuracy on training set is : {}".format(trainingScore/iteration))
print("Accuracy on test set is : {}".format(testingScore/iteration))
print(classification_report(yTest, yPredict,digits=3))

Results of RandomForest
Accuracy on training set is : 0.9999860188745191
Accuracy on test set is : 0.9565851670741652
              precision    recall  f1-score   support

         0.0      0.964     0.989     0.976    108110
         1.0      0.889     0.738     0.807     14200
         2.0      0.000     0.000     0.000       390

    accuracy                          0.957    122700
   macro avg      0.618     0.576     0.594    122700
weighted avg      0.952     0.957     0.953    122700



### Comparing all models

In [75]:
dict_results = batch_classify(X_train, Y_train, X_test, Y_test)
display_dict_models(dict_results)

trained Gradient Boosting Classifier in 43.73 s
trained Random Forest in 2.91 s
trained Logistic Regression in 0.30 s
trained Nearest Neighbors in 0.16 s
trained Decision Tree in 1.27 s


,classifier,train_score,test_score,train_time
1,Random Forest,1.000000,0.955990,2.905287
0,Gradient Boosting Classifier,0.998602,0.942950,43.731056
2,Logistic Regression,0.968193,0.941320,0.303719
3,Nearest Neighbors,0.951416,0.938060,0.160129
4,Decision Tree,1.000000,0.914425,1.270188
